# TODO title

## これなに?

TODO

# 分析対象

例としてなにか良いものがないか、`bigquery-public-data` から探してみると、`google_analytics_sample`という、`GA`のトランザクションのdatasetがあったので、これで、クリックする確率を予測してみようと思います

In [ ]:
!bq ls --project_id bigquery-public-data

# 準備

## datasetを生成

In [ ]:
!bq --locaton=eu mk google-analytics

datasetを作成します。今回は `bigquery-public-data` を使う都合上、locationを `EU`に指定します

## magic command

せっかくjupyterを使うので、magic commandを使用してbqに接続できるようにします。

In [4]:
!pip install google-cloud-bigquery

In [24]:
from google.cloud.bigquery import magics
from google.oauth2 import service_account
credentials = (service_account.Credentials.from_service_account_file('/Users/hiroaki/Downloads/scalable-data-science-b045d3ddcc4c.json'))
magics.context.credentials = credentials


In [20]:
PROJECT = 'scalable-data-science'

# %%bigqueryの使い方について

In [45]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


magic comannd `%%bigquery` を使用すると、cell内でbqのクエリを実行でき、更に返り値を`pandas.DataFrame`オブジェクトで受け取れるので、jupyter上で可視化などを行う際に非常に便利です。  
使い方としては以下のような形です

```
%%bigquery [返り値を格納する変数名]　--project [project_id] --param [クエリ内で使いたい変数を格納したdictまたはjson]  

[実行するクエリ]
```

具体的な実行はこのようになります

In [73]:
PARAMS = {'LIMIT' : 5}
PROJECT ='scalable-data-science'

In [82]:
%%bigquery data --project $PROJECT  --params $PARAMS
SELECT * FROM  `bigquery-public-data.google_analytics_sample.ga_sessions_20160801` LIMIT @LIMIT

In [77]:
data.head(2)

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,None,1,1470117657,1470117657,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",7194065619159478122,None,Social,Not Socially Engaged
1,None,151,1470083489,1470083489,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",8159312408158297118,None,Social,Not Socially Engaged


基本は、これだけで良いと思いますが、より詳細なオプションは[公式ドキュメント](https://google-cloud.readthedocs.io/en/latest/bigquery/generated/google.cloud.bigquery.magics.html)を参考にしてください。

# tableの概要を確認

In [94]:
%%bigquery --project $PROJECT 
SELECT table_name, column_name, data_type  FROM `bigquery-public-data.google_analytics_sample.INFORMATION_SCHEMA.COLUMNS` 
WHERE table_name ='ga_sessions_20160801'

,table_name,column_name,data_type
0,ga_sessions_20160801,visitorId,INT64
1,ga_sessions_20160801,visitNumber,INT64
2,ga_sessions_20160801,visitId,INT64
3,ga_sessions_20160801,visitStartTime,INT64
4,ga_sessions_20160801,date,STRING
5,ga_sessions_20160801,totals,"STRUCT<visits INT64, hits INT64, pageviews INT..."
6,ga_sessions_20160801,trafficSource,"STRUCT<referralPath STRING, campaign STRING, s..."
7,ga_sessions_20160801,device,"STRUCT<browser STRING, browserVersion STRING, ..."
8,ga_sessions_20160801,geoNetwork,"STRUCT<continent STRING, subContinent STRING, ..."
9,ga_sessions_20160801,customDimensions,"ARRAY<STRUCT<index INT64, value STRING>>"


# 学習に使用する変数のみを取り出す

このtableに入っている変数を

In [ ]:
bq show  --project_id bigquery-public-data google_analytics_sample.ga_sessions_20160801

で確認して見ます  
全部使うと、流石に面倒なので、予測のための説明変数は

TODO   
としましょう  
細かいcolumnの説明は[こちら](https://support.google.com/analytics/answer/3437719?hl=ja)にあるので、気になる人は御覧ください

目的変数である`isClick` はかなり入れ込んでるので、少し面倒そうですね

# 必要な変数が入っているcolumnのみを取り出す

In [241]:
%%bigquery --project $PROJECT 
-- 使用する変数が入っている要素のみを抽出
WITH data AS(
SELECT fullVisitorId, visitStartTime, hits, device 
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801` 
),
data2 AS(
SELECT fullVisitorId, hit.hour, hit.product, device.browser  FROM data, UNNEST(hits) AS hit
),
data3 AS(
-- TODO IF(product.isClick is null, False, True) AS isClick 
SELECT fullVisitorId, hour, browser, product.productPrice, product.isClick FROM data2, UNNEST(product) AS product 
)

SELECT  * FROM data3 LIMIT 10


,fullVisitorId,hour,browser,productPrice,isClick
0,7194065619159478122,23,Internet Explorer,25000000,None
1,7194065619159478122,23,Internet Explorer,50000000,None
2,7194065619159478122,23,Internet Explorer,100000000,None
3,7194065619159478122,23,Internet Explorer,250000000,None
4,8159312408158297118,13,Chrome,16990000,None
5,8159312408158297118,13,Chrome,16990000,None
6,8159312408158297118,13,Chrome,16990000,None
7,8159312408158297118,13,Chrome,16990000,None
8,8159312408158297118,13,Chrome,16990000,None
9,8159312408158297118,13,Chrome,16990000,None


いちいち上のクエリを叩くのも面倒なので、最初に作成したdatasetの中に保存します

In [ ]:
TODO 